In [109]:
# On-the fly reloads of libraries
%load_ext autoreload
%autoreload 2

# Import libraries
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

# Local modules
from Visualization import plot_stellar
from StarDetection import extract_stars
from FieldAnalysis import get_image_area_physical, get_diagonal_distance, get_pixel_scale, deg_to_px
from Solver import solve_field
from IndexHandler import get_index
from Visualization import compare_hashes
from Utilities import gaia_asterism_to_coords, local_asterism_to_coords

# Parameters
from dotenv import load_dotenv
import os
load_dotenv()
FIELD_DENSITY = float(os.getenv("FIELD_DENSITY"))

# Plotting setting
import PyQt5
%matplotlib qt5

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Get data

In [276]:
helix = fits.getdata('../Data/Helix.fits')
helixhdul = fits.open('../Data/Helix.fits')
plot_stellar(helix,stretch=False)

(<Figure size 3420x1876 with 1 Axes>, <Axes: >)

: 

### Examine the field

In [244]:
# Get field area
field_area = get_image_area_physical(fits.open('../Data/Helix.fits'))
print('Field area: {:.2f} square degrees'.format(field_area))

# This corresponds to the number of stars in the field
n_stars = FIELD_DENSITY * field_area
print('Utilized number of stars: {:.0f}'.format(n_stars))

pixel_scale = get_pixel_scale(helixhdul)

Field area: 0.26 square degrees
Utilized number of stars: 26


### Run star extraction

In [250]:
stars = extract_stars(helix)

In [163]:
# Plot the stars
fig,ax = plot_stellar(helix,stretch=True,vlims=[3,99.75])
for star in stars:
    ax.add_artist(plt.Circle((star['x'],star['y']),radius=np.median(star['fwhm'])*1.5,color='r',fill=False))

# Highlight the n_stars brightest stars
brightest_stars = stars[np.argsort(stars['flux'])[-int(n_stars):]]
ax.scatter(brightest_stars['x'],brightest_stars['y'],marker='x',color='g',s=10)

In [275]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming you have an array of standard deviations
std_deviations = stars['fwhm'] / (2 * np.sqrt(2 * np.log(2))) * pixel_scale * 3600 # Arcseconds per deg
mean_fwhm = np.mean(stars['fwhm']) * pixel_scale * 3600

# Generate some x values
x_values = np.linspace(0.75, 1.65, 1000)

# Calculate the mean and standard error of the standard deviations
mean_std = np.mean(std_deviations)
std_error = np.std(std_deviations, ddof=1) / np.sqrt(len(std_deviations))  # Standard error

# Calculate the upper and lower bounds for the 1 sigma error band with propagated error
upper_bound = (1 / (np.sqrt(2 * np.pi) * mean_std)) * np.exp(-0.5 * ((x_values - mean_std) / (std_error + std_deviations.std())) ** 2)
lower_bound = (1 / (np.sqrt(2 * np.pi) * mean_std)) * np.exp(-0.5 * ((x_values - mean_std) / (std_error - std_deviations.std())) ** 2)

# Calculate the mean of upper and lower bounds to ensure symmetry around the mean Gaussian
mean_upper_lower = (upper_bound + lower_bound) / 2

# Plotting
fig,ax = plt.subplots(tight_layout=True)
ax.plot(x_values, mean_upper_lower, color='k', label='Mean Gaussian')
ax.fill_between(x_values, upper_bound, lower_bound, alpha=0.5, label='1 sigma error band')
ax.legend()
ax.set_xlabel(r'$\sigma$ (Arcseconds)')
ax.set_ylabel('Probability density')
ax.set_title('Mean stellar profile')
# Highlight FWHM

Text(0.5, 1.0, 'Mean stellar profile')

In [272]:
mean_fwhm

2.821829808018934

### Let's examine some star shapes

In [21]:
get_diagonal_distance(helixhdul)

0.7262876325625859

### Let's try to solve the field

In [95]:
gaia_stars = get_index(helixhdul,299,22.3)

Using cached GAIA results.


In [96]:
from AsterismBuilder import build_asterisms_from_GAIA
asterisms,hashes_GAIA = build_asterisms_from_GAIA(gaia_stars,visualize=True)

#### Build asterims of input image

In [97]:
from AsterismBuilder import build_asterisms_from_input_image
asterisms,hashes_IMAGE = build_asterisms_from_input_image(helixhdul,visualize=True)

In [98]:
# Compare asterism to hash
fig,ax = plt.subplots(1,2,figsize=(10,5))
asterism = asterisms[0]
hash = hashes_IMAGE[0]
xs = np.array([asterism['xa'],asterism['xc'], asterism['xd'], asterism['xb']])
ys = np.array([asterism['ya'],asterism['yc'], asterism['yd'], asterism['yb']])

xs_hash = np.array([0,hash['xc'],hash['xd'],1])
ys_hash = np.array([0,hash['yc'],hash['yd'],1])

ax[0].scatter(xs,ys,color='r')
ax[1].scatter(xs_hash,ys_hash,color='b')

order = ['a','c','d','b']

for (x,y,letter) in zip(xs,ys,order):
    ax[0].annotate(letter,(x,y),color='r',fontsize=20)
    
for (x,y,letter) in zip(xs_hash,ys_hash,order):
    ax[1].annotate(letter,(x,y),color='b',fontsize=20)

### With the full solve_field code:

In [154]:
wcs,score,gaia_asterism,local_asterism,hash_gaia,hash_local = solve_field(helixhdul,299,22.3)

Using cached GAIA results.


In [155]:
wcs

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN' 'DEC--TAN' 
CRVAL : 299.8365765541068 22.61438094588813 
CRPIX : 1942.530517578125 1105.3917236328125 
PC1_1 PC1_2  : 0.9789031629427002 -0.20432473560677206 
PC2_1 PC2_2  : 0.20432473560677206 0.9789031629427002 
CDELT : 0.000121277460024584 0.000198407287986725 
NAXIS : 0  0

In [156]:
xs = wcs.world_to_pixel_values(gaia_stars['ra'],gaia_stars['dec'])[0]
ys = wcs.world_to_pixel_values(gaia_stars['ra'],gaia_stars['dec'])[1]

In [158]:
# Plot the GAIA stars and the picture with the new WCS
fig,ax = plot_stellar(helix,skycoords=False,stretch=True,vlims=[3,99.75],dpi=100)
ax.scatter(xs,ys,marker='o',color='g',s=10)

coords_local = local_asterism_to_coords(local_asterism)
coords_gaia = gaia_asterism_to_coords(gaia_asterism)
coords_gaia_pixels = wcs.world_to_pixel_values(coords_gaia['ra'],coords_gaia['dec'])
ax.scatter(coords_gaia_pixels[0],coords_gaia_pixels[1],marker='x',color='r',s=100)
ax.scatter(coords_local['x'],coords_local['y'],marker='x',color='b',s=100)
